In [1]:
import os, sys
sys.path.append(os.getcwd() + '/../..')
sys.path.append(os.getcwd() + '/..')

In [2]:
### allows to direct changes in the packages to be immidiately visibe in the notebook 
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime 
from multiprocessing.dummy import Pool as ThreadPool
from pandas_datareader._utils import RemoteDataError
from datetime import date

In [4]:
import packages.extract_data as extd

### Pull the famous Indices

In [7]:
indices_symbols = extd.get_famous_symbols(fresh=True)
indices_symbols = indices_symbols.drop_duplicates(subset='Symbol', keep="first")

In [8]:
indices_symbols

Symbol               Name     Index
0       RDSB.L  Royal Dutch Shell  FTSE 100
1       RDSA.L  Royal Dutch Shell  FTSE 100
2       HSBA.L      HSBC Holdings  FTSE 100
3       ULVR.L           Unilever  FTSE 100
4         BP.L                 BP  FTSE 100
...        ...                ...       ...
1347   TRMK.ME    Trubnaya Metall      MOEX
1348  MTLRP.ME           Mechel P      MOEX
1349   UWGN.ME            Npk Ovk      MOEX
1350   MTLR.ME             Mechel      MOEX
1351   LSRG.ME     Gruppa LSR PAO      MOEX

[1352 rows x 3 columns]

In [9]:
indices_symbols.shape

(1277, 3)

In [14]:
def get_price_history(symbol_list):
    today = date.today()
    df_history = pd.DataFrame()
    dt_start = '2015-01-01'
    #dt_end = '2020-04-30'
    dt_end = today.strftime('%Y-%m-%d')
    try:
        print('Pulling: ' + symbol_list)
        df_history = extd.get_individual_quote_history(symbol_list, dt_start, dt_end, '1d')
        if df_history is None:
            return None
        df_history['Symbol'] = symbol_list
    except RemoteDataError:
        exp_text = 'Symbol Not Found:' + symbol_list
        print(exp_text)
        error_q.put(exp_text)        
        pass 
    return df_history

### Run with Multithreading
##### multiprocessing will not help much in this case.

In [19]:
%%time

from queue import Queue
error_q = Queue()
### run 200 threads in parallel, each thread will pull prices for one symbol
pool = ThreadPool(200)
# Open the urls in their own threads and return the results
results = pd.concat(pool.map(get_price_history, indices_symbols.Symbol.values[0:5]))
# Close the pool and wait for the work to finish
pool.close()
pool.join()

Pulling: RDSB.LPulling: RDSA.LPulling: HSBA.LPulling: ULVR.L

Pulling: BP.L


Wall time: 1.05 s


In [20]:
results.head()

Open    High     Low   Close     Volume    Adj_Close  \
Datetime                                                                      
2015-01-02 09:00:00  2242.0  2256.0  2205.0  2235.5  2963987.0  1559.451416   
2015-01-05 09:00:00  2220.0  2229.0  2123.0  2128.5  5908429.0  1484.810181   
2015-01-06 09:00:00  2125.0  2172.5  2077.5  2121.0  5231824.0  1479.578003   
2015-01-07 09:00:00  2122.0  2181.5  2104.0  2148.5  5989533.0  1498.761475   
2015-01-08 09:00:00  2187.5  2207.5  2162.0  2200.0  4272088.0  1534.687256   

                     Symbol  
Datetime                     
2015-01-02 09:00:00  RDSB.L  
2015-01-05 09:00:00  RDSB.L  
2015-01-06 09:00:00  RDSB.L  
2015-01-07 09:00:00  RDSB.L  
2015-01-08 09:00:00  RDSB.L

In [18]:
results.index.max()

Timestamp('2020-11-06 09:00:00')